In [1]:
import os
import pickle
file_path = "vector_store.pkl"
file_name="paper.pdf"
vector_file_path = os.path.join("vectorStore",str.replace(file_name, 'pdf','pkl'))

In [7]:
vector_file_path

'vectorStore\\paper.pkl'

In [8]:
model="smollm"
from langchain_ollama import OllamaLLM
model=OllamaLLM(model=model)

In [9]:
model.invoke("tell me a joke")

"Here's a joke for you: Why did the computer go to the doctor? It had a virus!"

In [10]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(file_name)
pages=loader.load_and_split()
print(f"the number of pages loaded from the document are: {len(pages)}\n\nContent of the first page:{'-'*20} \n\n{pages[0]}")

the number of pages loaded from the document are: 9

Content of the first page:-------------------- 

page_content='RRCSNet:Recurrent Residual Convolutional Neural  
Network based cloud segmentation from satellite  
images  
 
 
 
 
 
 
 
 
 
 
 
 
Abstract— A crucial role in remote sensing applications,  
particularly in satellite imagery analysis, is played by cloud  
segmentation.  In this paper,  RRCSNet,  a novel  Recurrent  
Residual Convolutional Neural Network designed for accurate  
and efficient  cloud  segmentation.  The architecture  combines  the 
strengths  of recurrent  neural  networks  and residual  
convolutional  neural  networks  and enhance  feature  
representation.  The model  learns  sequential  patterns  in satellite  
data through recurrent connections, and residual connections  
help train  deeper  networks  by addressing  the vanishing  gradient  
problem.  Experimental  results  on Landsat  satellite  datasets  
prove the higher performance of RRCSNet compa

In [11]:
from langchain.prompts import PromptTemplate

template = """ Answer the question asked on the basis of the context provided.
If you can not find the answer then just say "answer is not available in the given text
context : {context}
Question : {question}
"""
prompt= PromptTemplate.from_template(template=template)
print(prompt.format(context="Here is the context", question="Here is the question"))


 Answer the question asked on the basis of the context provided.
If you can not find the answer then just say "answer is not available in the given text
context : Here is the context
Question : Here is the question



In [12]:
chain =prompt | model
chain.input_schema.model_json_schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [13]:
print(chain.invoke({ "context":"the name I was given is anisha", "question":"What is my name?"}))

The name I was given is Anisha


In [14]:
from langchain_ollama import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="smollm")

In [15]:

from langchain_community.vectorstores import DocArrayInMemorySearch
# if os.path.exists(vector_file_path):
#     with open(vector_file_path, "rb") as f:
#         vector_store = pickle.load(f)   
# else:
vector_store=DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
    # with open(vector_file_path, "wb") as f:
    #     pickle.dump(vector_store, f)

e:\projects\local_rag\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


PicklingError: Can't pickle <class 'docarray.index.abstract.InMemoryExactNNIndex[DocArrayDoc]'>: attribute lookup InMemoryExactNNIndex[DocArrayDoc] on docarray.index.abstract failed

In [ ]:
retriever=vector_store.as_retriever()
retriever.invoke("What is the result of the study")

In [10]:
from operator import itemgetter

chain = ({
    "context":itemgetter("question") | retriever,
    "question":itemgetter("question")
    }|
    prompt|model
    )

In [ ]:
print(chain.invoke({"question":"What is the result"}))

In [ ]:
questions=[
    "What is the title of the paper", 
    "How many authors are there in the paper",
    "what is the plateform used to run the experiment", 
    "summerize the methodology"
]

for question in questions:
    print(f"question: {question}")
    print(f"Answer:{chain.invoke({'question':question})}")
    print()